# Database

In [1]:
import src
from src import *
from src.creation import create_entry, combine_sites, NoModulator

In [2]:
db.init('database.db')
src.allodb.save_cifs = True

In [3]:
import pandas as pd
pd.DF = pd.DataFrame

# Data in GRALL

Data is based on the molecules/ligands, and only the entries with `Level of Confidence` of 1 and 2 have structural evidence (i.e., PDB structures to be included in the database), either in (1) Glycine receptor subunit alpha-1 or alpha-3, or in (2) related receptors of the ligand-gated ion channel (LGIC) family.

Therefore, all Uniprots of the LGIC family with an annotation score of 5/5 are going to be retrieved, and used to query the PDB database for structures that contain both a LGIC Uniprot protein chain and a molecule that matches one of the SMILES of the database for which structural data is annotated as available (Level of Confidence 1 or 2).

In [4]:
df = pd.read_csv("grall-v3.0-website-4-download.csv", na_values="N.A", keep_default_na=True)
df

,Name,SMILES,Family,Binding Site,Level of Confidence,Effect α1,Effect α3,Activity Type α1,Activity Value α1,Activity Type α3,Activity Value α3,doi,Kd_Resting(uM),Kd_active(uM),beta
0,conivaptan,Cc1nc2c([nH]1)CCN(C(=O)c1ccc(NC(=O)c3ccccc3-c3...,HTS,NaN,NaN,0,NaN,EC50(uM),Inactive,NaN,NaN,10.1021/jm501873p,NaN,NaN,NaN
1,PNU-120596,COc1cc(OC)c(NC(=O)Nc2cc(C)on2)cc1Cl,HTS,NaN,NaN,0,NaN,EC50(uM),Inactive,NaN,NaN,10.1021/jm501873p,NaN,NaN,NaN
2,pimozide,O=c1[nH]c2ccccc2n1C1CCN(CCCC(c2ccc(F)cc2)c2ccc...,HTS,NaN,NaN,+,NaN,EC50(uM),1.7,NaN,NaN,10.1021/jm501873p,NaN,NaN,NaN
3,cholecalciferol,C=C1CC[C@H](O)C/C1=C/C=C1\CCC[C@@]2(C)[C@H]1CC...,HTS,NaN,NaN,+,NaN,EC50(uM),0.4,NaN,NaN,10.1021/jm501873p,NaN,NaN,NaN
4,cinacalcet,C[C@@H](NCCCc1cccc(C(F)(F)F)c1)c1cccc2ccccc12,HTS,NaN,NaN,+,NaN,EC50(uM),0.32,NaN,NaN,10.1021/jm501873p,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Maksay2004-4e,Nc1cccc(C(=O)O[C@@H]2C[C@@H]3CC[C@H](C2)N3)c1,tropeine,(LA)-tropeine,5.0,-,-,Kd(uM),0.06,Kd(uM),0.06,10.1021/jm040814g,0.058479,0.081871,1.40
216,Maksay2004-2k,CN1[C@H]2CC[C@@H]1C[C@H](OC(=O)c1cc(Cl)cc3c1OC...,tropeine,NaN,NaN,+,+,Kd(uM),0.01,Kd(uM),0.01,10.1021/jm040814g,0.232809,0.013969,0.06
217,Maksay2004-4d,CC1(C)Cc2cc(Cl)cc(C(=O)O[C@@H]3C[C@@H]4CC[C@H]...,tropeine,NaN,NaN,+,+,Kd(uM),Inactive,Kd(uM),Inactive,10.1021/jm040814g,0.018113,0.002355,0.13
218,Maksay2004-9,CCN1[C@H]2CC[C@@H]1C[C@@H](OC(=O)c1ccccc1)C2,tropeine,NaN,NaN,+,+,Kd(uM),0.4,Kd(uM),0.4,10.1021/jm040814g,0.729458,0.401202,0.55


In [5]:
df_allo = df.loc[((df["Level of Confidence"] <= 2) & (df["Binding Site"] != "orthosteric"))]
df_allo

,Name,SMILES,Family,Binding Site,Level of Confidence,Effect α1,Effect α3,Activity Type α1,Activity Value α1,Activity Type α3,Activity Value α3,doi,Kd_Resting(uM),Kd_active(uM),beta
44,ethanol,CCO,alcohol,alcohol,2.0,+,+,EC50(uM),50000,NaN,NaN,10.1016/j.phrs.2015.07.002,NaN,NaN,NaN
57,ivermectin-b1a,CC[C@H](C)[C@H]1O[C@]2(CC[C@@H]1C)C[C@@H]1C[C@...,avermectin,ivermectin,1.0,+,NaN,EC50(uM),1.7,NaN,NaN,10.1074/jbc.M110.107789,NaN,NaN,NaN
115,Pregnenolone-sulfate,CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](OS(=O)...,neurosteroid,(-)-neurosteroid,2.0,-,NaN,Ki(uM),1.9,NaN,NaN,10.1016/S0028-3908(01)00071-5,NaN,NaN,NaN
116,THDOC,C[C@]12CC[C@@H](O)C[C@@H]1CC[C@@H]1[C@@H]2CC[C...,neurosteroid,(+)-neurosteroid,2.0,+,NaN,Maximum_effect(%),161,NaN,NaN,10.1113/JP272122,NaN,NaN,NaN
133,picrotin,CC(C)(O)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[...,picrotoxin,pore,1.0,-,-,IC50(uM),5.2,IC50(uM),6,10.1111/j.1471-4159.2007.04850.x,NaN,NaN,NaN
134,picrotoxinin,C=C(C)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[C@...,picrotoxin,pore,1.0,-,-,IC50(uM),2.1,IC50(uM),0.43,10.1111/j.1471-4159.2007.04850.x,NaN,NaN,NaN
136,propofol,CC(C)c1cccc(C(C)C)c1O,propofol,alcohol,2.0,+,NaN,EC50(uM)a1b,12.5,NaN,NaN,10.1213/01.ANE.0000120083.10269.54,19.0,10.26,0.54
142,AM-3607,C[C@H]1[C@H]2C(=O)N(C)c3ccncc3[C@H]2CN1S(=O)(=...,sulfonamide,top_ECD,1.0,+,+,EC50(uM),0.03,EC50(uM),0.05,10.1038/nsmb.3329,NaN,NaN,NaN
192,granisetron,CN1[C@H]2CCC[C@@H]1C[C@H](NC(=O)c1nn(C)c3ccccc...,tropeine,(LA)-tropeine,2.0,-,-,Kd(uM),12.5,Kd(uM),12.5,10.1016/S0028-3908(02)00213-7,12.5,37.50,3.00


In [6]:
import os, pickle, requests, json

##### Get Uniprots of LGIC family

Annotation score 5/5

In [8]:
file = "LGIC_uniprots.json"

if not os.path.isfile(file):
    lgic_uniprots = requests.get(
            "https://rest.uniprot.org/uniprotkb/stream?format=list&query=%28%28family%3A%22ligand-gated+ion+channel+%28TC+1.A.9%29+family%22%29%29+AND+%28annotation_score%3A5%29"
        ).content.decode().split("\n")

    with open(file, "wb") as f:
        pickle.dump(lgic_uniprots, f)

with open(file, "rb") as f:
    lgic_uniprots = pickle.load(f)
    
lgic_uniprots

['A0A0D9R3G4',
 'A0A0D9REP3',
 'A0A0P6K2L9',
 'A0A286XUB1',
 'A0A2I3MZA7',
 'A0A2K5CQD9',
 'A0A2K5MSW4',
 'A0A2K5SCY3',
 'A0A2K5U752',
 'A0A2K6CE69',
 'A0A2K6GC14',
 'A0A2K6GGC7',
 'A0A2K6KXI8',
 'A0A2K6U733',
 'A0A2R8MV58',
 'A0A2R9CBB5',
 'A0A2Y9EU19',
 'A0A337SQQ4',
 'A0A3Q7R9T2',
 'A0A3Q7UG22',
 'A0A452EZE1',
 'A0A452T828',
 'A0A452VFF0',
 'A0A493U029',
 'A0A4W2DFT2',
 'A0A4X1TUM3',
 'A0A4X1U8B8',
 'A0A4X2KCK0',
 'A0A6I8QF26',
 'A0A8B7EN39',
 'A0A8C0QAY2',
 'A0A8C8WAS3',
 'A0A8C8WIX8',
 'A0A8D0GJ56',
 'A0A8I3MZA4',
 'A0A8I3PAT2',
 'A0A8I6A8K2',
 'A0A8J8YBP3',
 'A5X5Y0',
 'A8MPY1',
 'D1LYT2',
 'E1JJR2',
 'F1RDZ6',
 'F1SNR6',
 'F6RM03',
 'F6UQM3',
 'F6YAL1',
 'F6ZNX7',
 'F7G2F3',
 'F7GTN8',
 'G1LMJ3',
 'G1NVY3',
 'G1S3B5',
 'G1SHV0',
 'G1SI02',
 'G3REP2',
 'G3T2T7',
 'G3TNL3',
 'G3W3E9',
 'G5EBR3',
 'G5ECJ0',
 'G5ECT0',
 'G5EG88',
 'G7P8R5',
 'H0VPE2',
 'H0WJ85',
 'H2PH52',
 'H2QRU5',
 'I3MQM4',
 'K7GDT5',
 'M3WNZ0',
 'M3YQE3',
 'M9PFD8',
 'O00591',
 'O14764',
 'O18276',
 'O70174',
 

##### Query the PDB

In [9]:
from rcsbsearchapi.search import AttributeQuery, ChemSimilarityQuery

In [10]:
file = "LGIC_pdbs.json"

if not os.path.isfile(file):
    pdb_query = {}
    
    for (name, smi) in df_allo[["Name", "SMILES"]].values.tolist():
        upquery = AttributeQuery(
            attribute="rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
            operator="in",
            negation=False,
            value=lgic_uniprots
        )
        smiquery = ChemSimilarityQuery(
            value=smi,
            query_type="descriptor",
            descriptor_type="SMILES",
            match_type="graph-relaxed" # To search for similar molecules to the SMILES
        )
    
        pdb_query[name] = (
            (name, smi),
            list((upquery & smiquery)()),
            list((upquery & smiquery)("mol_definition"))
        )

    with open(file, "wb") as f:
        pickle.dump(pdb_query, f)

with open(file, "rb") as f:
    pdb_query = pickle.load(f)
    
pdb_query

{'ethanol': (('ethanol', 'CCO'), ['4HFE'], ['EOH']),
 'ivermectin-b1a': (('ivermectin-b1a',
   'CC[C@H](C)[C@H]1O[C@]2(CC[C@@H]1C)C[C@@H]1C[C@@H](C/C=C(\\C)[C@@H](O[C@H]3C[C@H](OC)[C@@H](O[C@H]4C[C@H](OC)[C@@H](O)[C@H](C)O4)[C@H](C)O3)[C@@H](C)/C=C/C=C3\\CO[C@@H]4[C@H](O)C(C)=C[C@@H](C(=O)O1)[C@]34O)O2'),
  ['3JAF', '5VDH', '5VDI', '6VM0', '6VM2', '6VM3', '8F4V', '8FE1', '8UZJ'],
  ['IVM']),
 'Pregnenolone-sulfate': (('Pregnenolone-sulfate',
   'CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](OS(=O)(=O)O)CC[C@]4(C)[C@H]3CC[C@]12C'),
  ['5OSC', '8SGO'],
  ['A8W']),
 'THDOC': (('THDOC',
   'C[C@]12CC[C@@H](O)C[C@@H]1CC[C@@H]1[C@@H]2CC[C@]2(C)[C@@H](C(=O)CO)CC[C@@H]12'),
  ['5OSB'],
  ['A8Z']),
 'picrotin': (('picrotin',
   'CC(C)(O)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[C@@]34O[C@@H]3C[C@]2(O)[C@@]14C'),
  [],
  []),
 'picrotoxinin': (('picrotoxinin',
   'C=C(C)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[C@@]34O[C@@H]3C[C@]2(O)[C@@]14C'),
  ['6HUG', '6HUJ', '6UD3', '6X40', '8OQ8', '8OQA'],
  ['RI5']),


In [11]:
[p for v in pdb_query.values() for p in v[1]]

['4HFE',
 '3JAF',
 '5VDH',
 '5VDI',
 '6VM0',
 '6VM2',
 '6VM3',
 '8F4V',
 '8FE1',
 '8UZJ',
 '5OSC',
 '8SGO',
 '5OSB',
 '6HUG',
 '6HUJ',
 '6UD3',
 '6X40',
 '8OQ8',
 '8OQA',
 '3P50',
 '5MUO',
 '5MUR',
 '5MVM',
 '5MVN',
 '5MZR',
 '6X3T',
 '5TIN',
 '5TIO',
 '5VDH',
 '5VDI',
 '6NP0']

In [12]:
len([p for v in pdb_query.values() for p in v[1]]), "PDBs"

(31, 'PDBs')

## Data already in the database

In [13]:
list(PDB.select().where(PDB.entry_id.in_([p.lower() for v in pdb_query.values() for p in v[1]])))

[<PDB: 3jaf>,
 <PDB: 3p50>,
 <PDB: 5muo>,
 <PDB: 5mur>,
 <PDB: 5mvn>,
 <PDB: 5osb>,
 <PDB: 5osc>,
 <PDB: 5vdh>,
 <PDB: 5vdi>]

In [14]:
len(PDB.select().where(PDB.entry_id.in_([p.lower() for v in pdb_query.values() for p in v[1]]))), "PDBs already in the database"

(9, 'PDBs already in the database')

In [15]:
[
    (
        p.entry_id, 
        list(s.modulator_residues.label_comp_id.unique().tolist() for s in p.sites),
        [(k, v[-1]) for k,v in pdb_query.items() if p.entry_id.upper() in v[-2]]
    )
    for p in list(PDB.select().where(PDB.entry_id.in_([p.lower() for v in pdb_query.values() for p in v[1]])))
    
]

[('3jaf', [['IVM']], [('ivermectin-b1a', ['IVM'])]),
 ('3p50', [['PFL']], [('propofol', ['PFL'])]),
 ('5muo', [['PFL']], [('propofol', ['PFL'])]),
 ('5mur', [['PFL']], [('propofol', ['PFL'])]),
 ('5mvn', [['PFL']], [('propofol', ['PFL'])]),
 ('5osb', [['A8Z']], [('THDOC', ['A8Z'])]),
 ('5osc', [['A8W']], [('Pregnenolone-sulfate', ['A8W'])]),
 ('5vdh', [['IVM']], [('ivermectin-b1a', ['IVM']), ('AM-3607', ['7C6'])]),
 ('5vdi', [['IVM']], [('ivermectin-b1a', ['IVM']), ('AM-3607', ['7C6'])])]

## Processing

In [16]:
processed = [pd.Series(index=df_allo.columns, dtype=object)]

In [17]:
errors = {}
error_entries = []

In [18]:
errors_groups = lambda: {
    val: sorted(
        k for k, v in errors.items()
        if any(map(lambda x: val in x, v))
    )
    for val in set(
        map(
            lambda x: x[0].split(": ")[-1], 
            errors.values()
        )
    )
}

In [19]:
def process_entry(entry, update={}, auto_site_grouping=True, stringent_site_grouping=True):
    try:
        pdb = update["pdb"]
        mod = update["mod"]
        print(pdb, mod)
    
        old_sites = None
        # If PDB already exists, save its site IDs
        if PDB.get_or_none(PDB.entry_id == pdb) is not None:
            old_sites = list(PDB.get(PDB.entry_id == pdb).sites)
            
        with db.atomic() as txn:
            sites = create_entry(db, pdb, mod, auto_site_grouping, stringent_site_grouping)
            
            assert len(sites.objects()) == 1, f"{pdb}, {mod}: using only residue name retrieves more than one site(group)"
            
            for site in sites.objects():
                site.info["source"] = {
                    "grall": [{
                        "entry": entry.to_dict(),
                        "version": '3.0',
                        "date": "12-3-2024",
                        "update": update
                    }],
                }
                site.save()
                
            if old_sites is not None:
                try:
                    combine_sites(db, pdb.lower(), old_sites, sites.objects(), auto_site_grouping, stringent_site_grouping)
                except Exception as e:
                    assert False, "combine_sites failed; " + str(e.args[0])

        return sites

    except (AssertionError, KeyError) as error:
        id = pdb.lower()
        errors.setdefault(id, [])
        errors[id].append(str(error.args[0]))
        error_entries.append(entry)
        print(id, errors[id])

    

In [20]:
pdb_query

{'ethanol': (('ethanol', 'CCO'), ['4HFE'], ['EOH']),
 'ivermectin-b1a': (('ivermectin-b1a',
   'CC[C@H](C)[C@H]1O[C@]2(CC[C@@H]1C)C[C@@H]1C[C@@H](C/C=C(\\C)[C@@H](O[C@H]3C[C@H](OC)[C@@H](O[C@H]4C[C@H](OC)[C@@H](O)[C@H](C)O4)[C@H](C)O3)[C@@H](C)/C=C/C=C3\\CO[C@@H]4[C@H](O)C(C)=C[C@@H](C(=O)O1)[C@]34O)O2'),
  ['3JAF', '5VDH', '5VDI', '6VM0', '6VM2', '6VM3', '8F4V', '8FE1', '8UZJ'],
  ['IVM']),
 'Pregnenolone-sulfate': (('Pregnenolone-sulfate',
   'CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](OS(=O)(=O)O)CC[C@]4(C)[C@H]3CC[C@]12C'),
  ['5OSC', '8SGO'],
  ['A8W']),
 'THDOC': (('THDOC',
   'C[C@]12CC[C@@H](O)C[C@@H]1CC[C@@H]1[C@@H]2CC[C@]2(C)[C@@H](C(=O)CO)CC[C@@H]12'),
  ['5OSB'],
  ['A8Z']),
 'picrotin': (('picrotin',
   'CC(C)(O)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[C@@]34O[C@@H]3C[C@]2(O)[C@@]14C'),
  [],
  []),
 'picrotoxinin': (('picrotoxinin',
   'C=C(C)[C@H]1[C@@H]2C(=O)O[C@H]1[C@H]1OC(=O)[C@@]34O[C@@H]3C[C@]2(O)[C@@]14C'),
  ['6HUG', '6HUJ', '6UD3', '6X40', '8OQ8', '8OQA'],
  ['RI5']),


In [21]:
for name, (smi, pdbs, comp) in pdb_query.items():
    if len(pdbs) > 0 and len(comp) > 0:
        entry = df_allo.query(f"Name == '{name}'").squeeze()
        assert len(comp) == 1
        for pdb in pdbs:
            process_entry(
                entry,
                {"pdb": pdb.lower(), "mod": [[{"auth_comp_id": comp[0]}]]},
                auto_site_grouping=True,
                stringent_site_grouping=True
            )

4hfe [[{'auth_comp_id': 'EOH'}]]
 ExecutiveLoad-Warning: No _chem_comp_bond data for residue 'UNL'
 ExecutiveLoad-Warning: No _chem_comp_bond data for residue 'UNL'
 ExecutiveLoad-Warning: No _chem_comp_bond data for residue 'UNL'
 ExecutiveLoad-Warning: No _chem_comp_bond data for residue 'UNL'
 ExecutiveLoad-Warning: No _chem_comp_bond data for residue 'UNL'
3jaf [[{'auth_comp_id': 'IVM'}]]
5vdh [[{'auth_comp_id': 'IVM'}]]
5vdi [[{'auth_comp_id': 'IVM'}]]
6vm0 [[{'auth_comp_id': 'IVM'}]]
6vm2 [[{'auth_comp_id': 'IVM'}]]
6vm3 [[{'auth_comp_id': 'IVM'}]]
8f4v [[{'auth_comp_id': 'IVM'}]]
8fe1 [[{'auth_comp_id': 'IVM'}]]
8fe1 ["8fe1, [[{'auth_comp_id': 'IVM'}]]: using only residue name retrieves more than one site(group)"]
8uzj [[{'auth_comp_id': 'IVM'}]]
5osc [[{'auth_comp_id': 'A8W'}]]
8sgo [[{'auth_comp_id': 'A8W'}]]
5osb [[{'auth_comp_id': 'A8Z'}]]
6hug [[{'auth_comp_id': 'RI5'}]]
6huj [[{'auth_comp_id': 'RI5'}]]
6ud3 [[{'auth_comp_id': 'RI5'}]]
6x40 [[{'auth_comp_id': 'RI5'}]]
8oq8 

### Error correction

In [22]:
errors

{'8fe1': ["8fe1, [[{'auth_comp_id': 'IVM'}]]: using only residue name retrieves more than one site(group)"]}

In [23]:
PDB.get_or_none(PDB.entry_id == "8fe1")

In [24]:
name = "ivermectin-b1a"
pdb = "8fe1"
comp = "IVM"

for chain in ["H", "R", "QA"]:
    process_entry(
        entry,
        {"pdb": pdb.lower(), "mod": [[{"auth_comp_id": comp, 
                                       "label_asym_id": chain}]]},
        auto_site_grouping=True,
        stringent_site_grouping=True
    )

8fe1 [[{'auth_comp_id': 'IVM', 'label_asym_id': 'H'}]]
8fe1 [[{'auth_comp_id': 'IVM', 'label_asym_id': 'R'}]]
8fe1 [[{'auth_comp_id': 'IVM', 'label_asym_id': 'QA'}]]


In [25]:
[s.modulator for s in PDB.get_or_none(PDB.entry_id == "8fe1").sites]

[{'label_asym_id': ['H']}, {'label_asym_id': ['R']}, {'label_asym_id': ['FA']}]

The pentamer contains a monomer of a different isoform than the rest of monomers that makes the sites of IVM different.

<br>

In [26]:
grall_sites = list(list(s.info["source"].keys()) for s in Site.select().where(Site.info["source"].contains("grall")))
grall_sites

[['grall'],
 ['allosteric_database', 'grall'],
 ['allosteric_database', 'grall'],
 ['allosteric_database', 'grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['allosteric_database', 'grall'],
 ['grall'],
 ['allosteric_database', 'grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['allosteric_database', 'grall'],
 ['allosteric_database', 'grall'],
 ['allosteric_database', 'grall'],
 ['grall'],
 ['allosteric_database', 'grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall'],
 ['grall']]

In [27]:
grall_sites.count(['allosteric_database', 'grall'])

9

In [28]:
grall_sites.count(['grall'])

24

# Statistics

In [29]:
# Total number of sites
len(Site.select())

3181

In [30]:
# Total number of different PDBs
len(PDB.select())

3041

In [31]:
# PDBs with no sites (expected 0)
[(pdb.entry_id, pdb) for p in PDB.select() if len(p.sites) == 0]

[]

In [32]:
# Number of sites in PDBs
set(len(p.sites) for p in PDB.select())

{1, 2, 3, 4}

In [33]:
# Modulator identifier fields used
set(tuple(s.modulator.keys()) for s in Site.select())

{('label_asym_id',)}

In [34]:
# Number of different entity instances (different "label_asym_id") annotated as modulators to look for outliers
modulator_chains = dict(
    sorted(
        {s.id: len(s.modulator["label_asym_id"]) for s in Site.select()}.items(),
        key=lambda i: i[-1], reverse=True
    )
)
modulator_chains

{520: 24,
 582: 8,
 608: 6,
 4806: 6,
 5088: 6,
 5246: 6,
 5320: 6,
 5365: 6,
 160: 4,
 187: 4,
 193: 4,
 368: 4,
 380: 4,
 412: 4,
 684: 4,
 741: 4,
 4493: 4,
 4767: 4,
 336: 3,
 419: 3,
 481: 3,
 482: 3,
 483: 3,
 552: 3,
 569: 3,
 570: 3,
 592: 3,
 599: 3,
 615: 3,
 624: 3,
 2319: 3,
 2327: 3,
 2334: 3,
 2633: 3,
 4542: 3,
 4559: 3,
 4577: 3,
 4590: 3,
 4636: 3,
 4733: 3,
 4834: 3,
 4842: 3,
 4863: 3,
 4886: 3,
 4907: 3,
 4929: 3,
 4949: 3,
 4966: 3,
 4979: 3,
 4992: 3,
 5002: 3,
 5003: 3,
 5122: 3,
 5143: 3,
 5190: 3,
 5200: 3,
 5267: 3,
 5272: 3,
 5382: 3,
 5463: 3,
 5471: 3,
 5474: 3,
 5: 2,
 14: 2,
 51: 2,
 75: 2,
 81: 2,
 91: 2,
 92: 2,
 93: 2,
 94: 2,
 95: 2,
 97: 2,
 136: 2,
 140: 2,
 144: 2,
 147: 2,
 150: 2,
 153: 2,
 168: 2,
 175: 2,
 198: 2,
 248: 2,
 254: 2,
 269: 2,
 272: 2,
 288: 2,
 318: 2,
 328: 2,
 339: 2,
 371: 2,
 383: 2,
 418: 2,
 432: 2,
 446: 2,
 463: 2,
 467: 2,
 480: 2,
 488: 2,
 491: 2,
 523: 2,
 526: 2,
 529: 2,
 530: 2,
 573: 2,
 585: 2,
 627: 2,
 748: 2,


In [35]:
db.close()

True